# load packages

In [2]:
import csv
import numpy as np
import pandas as pd 
import requests
from tqdm.auto import tqdm
import os
import time

# get all images
save them in ./images_all_2/[country]/[year]/[set_id]/[number].jpg
And save this patch in new column [Image_Path]

In [3]:
# load csv
df = pd.read_csv('./contents_merged/all_contents_updated_3.csv')

C:\Users\Robin\AppData\Local\Temp\ipykernel_22508\3062245543.py:2: DtypeWarning: Columns (12,13,14,15,16,21,27,28,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./contents_merged/all_contents_updated_3.csv')


In [ ]:
max_retries = 3
retry_delay = 2  # seconds

# Assuming df is your DataFrame
batch_size = 10000
num_batches = int(np.ceil(len(df) / batch_size))

# set starting batch because some batches were already processed
start_batch = 91

for batch_number in tqdm(range(start_batch, num_batches), desc="Processing Batches", leave=False):
    start_index = batch_number * batch_size
    end_index = min((batch_number + 1) * batch_size, len(df))
    df_batch = df.iloc[start_index:end_index].copy()

    for index, row in tqdm(df_batch.iterrows(), total=df_batch.shape[0], desc=f"Processing Batch {batch_number}", leave=False):

        if str(row['Image_URL']) == 'nan':
            # set to None
            df_batch.loc[index, 'Image_Path'] = None
            continue

        try:
            url = row['Image_URL']
            url = "https://www.stampworld.com/" + str(url)
            for attempt in range(max_retries):
                try:
                    response = requests.get(url)
                    response.raise_for_status()  # Raises an error for bad responses
                    break  # Exit the loop if the request is successful
                except requests.exceptions.RequestException as e:
                    if attempt < max_retries - 1:
                        time.sleep(retry_delay)  # Wait before retrying
                    else:
                        raise  # Re-raise the exception if the last attempt fails

            country = row['Country']
            year = row['Year']
            set_id = row['Set_ID']
            # split url at / and get last element
            filename = url.split('/')[-1]

            if response.status_code == 200:
                path_to_store_image = f"./images_all_2/{country}/{year}/{set_id}/{filename}"
                # create folder if it doesn't exist
                if not os.path.exists(os.path.dirname(path_to_store_image)):
                    os.makedirs(os.path.dirname(path_to_store_image))
                with open(path_to_store_image, 'wb') as f:
                    f.write(response.content)
                # save path to new column
                df_batch.loc[index, 'Image_Path'] = path_to_store_image
            else:
                raise Exception(f"Error 1 at index {index} with url: {url} (in batch {batch_number})")

        except Exception as e:
            print(f"Error 2 at index {index} with url: {url}, set value to None (in batch {batch_number})")
            print(e)

            # set to None
            df_batch.loc[index, 'Image_Path'] = None


            # raise exception
            # raise e

    # Save the batch DataFrame to a CSV file
    df_batch.to_csv(f'./contents_merged/batches_with_images/batch_{batch_number}.csv', index=False)

Processing Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Processing Batch 91:   0%|          | 0/10000 [00:00<?, ?it/s]

Error 2 at index 910000 with url: https://www.stampworld.com//media/catalogue/South-Vietnam/Postage-stamps/BX-i.jpg, set value to None (in batch 91)
[WinError 433] Ein nicht vorhandenes Gerät wurde angegeben: './images_all_2'
Error 2 at index 910001 with url: https://www.stampworld.com//media/catalogue/South-Vietnam/Postage-stamps/BX1-i.jpg, set value to None (in batch 91)
[WinError 433] Ein nicht vorhandenes Gerät wurde angegeben: './images_all_2'
Error 2 at index 910002 with url: https://www.stampworld.com//media/catalogue/South-Vietnam/Postage-stamps/BX2-i.jpg, set value to None (in batch 91)
[WinError 433] Ein nicht vorhandenes Gerät wurde angegeben: './images_all_2'
Error 2 at index 910003 with url: https://www.stampworld.com//media/catalogue/South-Vietnam/Postage-stamps/BX3-i.jpg, set value to None (in batch 91)
[WinError 433] Ein nicht vorhandenes Gerät wurde angegeben: './images_all_2'
Error 2 at index 910005 with url: https://www.stampworld.com//media/catalogue/South-Vietnam/P

KeyboardInterrupt: 